## 1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.


In [ ]:
import os
import json
import pickle

import numpy as np
import pandas as pd
import scipy
from scipy.stats import kendalltau, spearmanr
import tqdm

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

PLAYERS_PATH = "./data/players.pkl"
RESULTS_PATH = "./data/results.pkl"
TOURNAMENTS_PATH = "./data/tournaments.pkl"
RESULTS_TRAIN_PATH = "./data/results_train.pkl"
RESULTS_TEST_PATH = "./data/results_test.pkl"
DATASET_TRAIN_PATH = "./data/commands_dataset_train.csv"
DATASET_TEST_PATH = "./data/commands_dataset_test.csv"

# read data
data_players = pd.DataFrame(pd.read_pickle(PLAYERS_PATH).values())
data_tours = pd.DataFrame(pd.read_pickle(TOURNAMENTS_PATH).values())

# tournaments train-test splitting
data_tours["dateStart"] = pd.to_datetime(data_tours.dateStart)
data_tours_train = data_tours[data_tours.dateStart.apply(lambda x: x.year) == 2019]
data_tours_test = data_tours[data_tours.dateStart.apply(lambda x: x.year) == 2020]

Возьмем данные игр. Оставим только те, в которых для **каждой** команды известна маска ответов. Таким образом отфильтруется несколько игр, но пока что оставим это, ведь там какая-то грязь в данных, неохота вникать

In [2]:
def is_valid_game(game):
    cond = (
        bool(game) and  # is not empty list
        all(['mask' in x.keys() for x in game]) and  # 'mask' attribute exist
        all([x['mask'] for x in game]) and  # all masks are valid
        len(set([len(x['mask']) for x in game])) == 1 and  # all masks lenghts are equal
        all([not bool(set(x['mask']).difference('10X')) 
             for x in game])  # all masks have only {1,0,X}    
    )
    return cond


def select_valid_games(data_tours_train, data_tours_test, first_data_reading=False):
    """
    select games with valid 'mask' attribute, it means that 
    'mask' exist for each command
    
    """
    if not os.path.exists(RESULTS_TRAIN_PATH) and not os.path.exists(RESULTS_TEST_PATH):
        first_data_reading = True
    
    if first_data_reading:
        data_result = pd.read_pickle(RESULTS_PATH)
        assert len(data_result) == len(data_tours)
        
        data_tours_train_test = [data_tours_train, data_tours_test]
        data_result_train_test = [[], []]
        
        for data_part in range(2):
            for game_id, game_type in data_tours_train_test[data_part][['id', 'type']].values:
                cur = data_result[game_id]
                if is_valid_game(cur):
                    for com in cur:
                        com['mask'] = com['mask'].replace('X', '')
                        com['game_type'] = game_type['id']
                        com['game_id'] = game_id
                    data_result_train_test[data_part].append(cur)
                    
        # check
        for _data in data_result_train_test:
            for x in _data:
                for y in x:
                    assert bool(y['mask']), repr(y)
                    
        data_result_train, data_result_test = data_result_train_test

        with open(RESULTS_TRAIN_PATH, 'wb') as fout:
            pickle.dump(data_result_train, fout)

        with open(RESULTS_TEST_PATH, 'wb') as fout:
            pickle.dump(data_result_test, fout)

    else:
        # load previously saved sub-datasets
        with open(RESULTS_TRAIN_PATH, 'rb') as fin:
            data_result_train = pickle.load(fin)

        with open(RESULTS_TEST_PATH, 'rb') as fin:
            data_result_test = pickle.load(fin)
            
    return data_result_train, data_result_test

## 2. Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:

- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [3]:
def split_data_by_que(data_result, test=False):
    """ Build by-question results for players independently """
    data = []
    prev_qid = 0

    for game in tqdm.tqdm(data_result):
        game_type = game[0]['game_type']
        game_id = game[0]['game_id']

        for team in game:
            # drop 'X'-answers
            mask = team['mask'].replace('X', '')
            # and replace '?'-answers by None
            mask = [int(x) if x in '10' else None for x in mask]
            team_id = team['team']['id']
            players = [x['player']['id'] for x in team['teamMembers']]
            for player_id in players:
                for qid_in_game, answer in enumerate(mask):
                    if answer is None:
                        continue
                    question_id = prev_qid + qid_in_game

                    data.append([
                        game_id,
                        game_type,
                        team_id,
                        player_id,
                        question_id,
                        qid_in_game,
                        answer,
                    ])

        prev_qid += len(mask)

    cols = ['game_id', 'game_type', 'team_id', 'player_id', 
            'question_id', 'qid_in_game', 'answer']
    data = pd.DataFrame(data, columns=cols)   
    return data

In [4]:
data_result_train, data_result_test = select_valid_games(
    data_tours_train, data_tours_test)

In [5]:
train_data = split_data_by_que(data_result_train)

print(train_data.shape)
train_data.head(2)

100%|██████████| 649/649 [00:15<00:00, 42.35it/s] 


(15692612, 7)


,game_id,game_type,team_id,player_id,question_id,qid_in_game,answer
0,4772,3,45556,6212,0,0,1
1,4772,3,45556,6212,1,1,1


In [ ]:
ohe = OneHotEncoder(dtype=np.int8, handle_unknown='ignore')
X_train = ohe.fit_transform(train_data[['player_id', 'question_id']])
y_train = train_data.answer.values.astype(np.int8)

In [7]:
print(X_train.shape, y_train.shape)

(15692612, 86026) (15692612,)


In [ ]:
logreg = LogisticRegression(random_state=22799)
logreg.fit(X_train, y_train)

Только что мы научили логрег предсказывать "правильной ответ" на основе силы игрока и сложности вопроса.

Поглядим на силу лучших игроков, поищем знакомые имена

In [ ]:
feature_names = ohe.get_feature_names(['player_id', 'question_id'])

baseline_importance = pd.DataFrame({
    'feature': feature_names,
    'value': logreg.coef_[0],
})
player_power = baseline_importance[
    baseline_importance.feature.str.startswith('player')]

_f = lambda s: int(s.split('_')[-1])
player_power['player_id'] = player_power.feature.apply(_f)

player_power = pd.merge(
    player_power, data_players, 
    left_on='player_id', right_on='id'
)
player_power = player_power.sort_values('value', ascending=False)

In [10]:
player_power.iloc[:20, [3,4,5,6,1]]

,id,name,patronymic,surname,value
3830,27403,Максим,Михайлович,Руссо,4.160863
601,4270,Александра,Владимировна,Брутер,4.030017
4019,28751,Иван,Николаевич,Семушин,3.979427
3902,27822,Михаил,Владимирович,Савченков,3.893555
4205,30152,Артём,Сергеевич,Сорожкин,3.811722
4225,30270,Сергей,Леонидович,Спешков,3.811201
2907,20691,Станислав,Григорьевич,Мереминский,3.686195
2552,18036,Михаил,Ильич,Левандовский,3.638140
3200,22799,Сергей,Игоревич,Николенко,3.561250
3650,26089,Ирина,Сергеевна,Прокофьева,3.556243


Видим, что в топе **Сергей Николенко**, Александр Либер, Иван Семушин, Сергей Спешков ...

Это должно что-то 

---------------

## 3. Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

- сначала предскажем повопросные вероятности правильных ответов
- потом посчитаем предсказанное количество правильных ответов от каждой команды в турнире и отсортируем в поисках ранга


- команда делает правильный ответ, если хотя бы 1 ее член правильно отвечает:
    
    $P_{command}(ans = 1) = 1 - \prod_{i \in players} (1 - P_i(ans = 1)) $
    
- а предсказанное количество верных можно найти, умножив данную вероятность на кол-во вопросов в турнире (но это не нужно):

    $E(numofcorrectans) = P_{command}(ans = 1) * N $
- предсказывать ранги команд можно по командной вероятности ответить на вопрос (для всех вопросов в игре она одинакова) 

In [11]:
def prepare_test():
    players_train = set(np.unique(train_data['player_id']))
    
    test = []
    for teams in tqdm.tqdm(data_result_test):
        tournament_id = teams[0]['game_id']
        for team in teams:
            team_id = team['team']['id']
            mask = list(map(int, team['mask']))
            players = [x['player']['id'] for x in team['teamMembers']]
            for player_id in players:
                # only players from train (Explicitly say that)
                if player_id not in players_train: 
                    continue

                # questions have no usefull index, give them -1
                test.append((tournament_id, team_id, player_id, 
                             -1, sum(mask))) 

    test = np.vstack(test).astype(np.int32)
    cols = ['tournament_id', 'team_id', 'player_id', 
            'question_id', 'n_correct']
    test = pd.DataFrame(test, columns = cols)
    return test

In [ ]:
test_data = prepare_test()
X_test = ohe.transform(test_data[['player_id', 'question_id']])

assert X_test[:, -train_data.question_id.nunique():].max() == 0, (
    'wrong transformation for question features'
)
y_preds = logreg.predict_proba(X_test)[:, 1]

In [13]:
def compute_scores(data, preds):
    data['pred'] = preds
    data['score'] = data.groupby(['tournament_id', 'team_id'])['pred']\
                    .transform(lambda x: 1 - np.prod(1 - x))
    rating = data[
        ['tournament_id', 'team_id', 'n_correct', 'score']
    ].drop_duplicates().reset_index(drop=True)
    
    # True command rating
    rating = rating.sort_values(by=['tournament_id', 'n_correct'], 
                                ascending=False)
    rating['real_rank'] = rating.groupby('tournament_id')['n_correct']\
                         .transform(lambda x: np.arange(1, len(x) + 1))
    
    # Predicted command rating
    rating = rating.sort_values(by=['tournament_id', 'score'], 
                                ascending=False)
    rating['pred_rank'] = rating.groupby('tournament_id')['score']\
                         .transform(lambda x: np.arange(1, len(x) + 1))

    rating = rating.astype(np.int32)
    
    score_spearman = rating.groupby('tournament_id').apply(
        lambda x: spearmanr(x['real_rank'], x['pred_rank']
                           ).correlation).mean()
    score_kendall = rating.groupby('tournament_id').apply(
        lambda x: kendalltau(x['real_rank'], x['pred_rank']
                            ).correlation).mean()
    
    print(f"Корреляция Спирмана: {score_spearman: .3f}")
    print(f"Корреляция Кендалла: {score_kendall: .3f}")

In [14]:
compute_scores(test_data, y_preds)

Корреляция Спирмана:  0.806
Корреляция Кендалла:  0.635


Адекватный результат, хотя первично выдавало >0.9   :D

-----------

## 4. Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


### Схема:
Мы знаем, что команда предскажет верно по этой формуле (дублирую ту, что выше):

$P_{command}(ans = 1) = 1 - \prod_{i \in players} (1 - P_i(ans = 1)) $
    
Формально, это же и есть сила команды.

А вклад игрока - это вероятность правильного ответа игрока, деленая на командную вероятность, отнормированная (clipped), чтобы быть в интервале [0, 1]:

$Power_i = clip(\frac{P_i(ans = 1)}{P_{command}(ans = 1)}) $

Т.е. Е-шаг будет представлять модификацию предсказываемого вектора по изложенному механизму, тепеь он будет содержать "командную" силу игроков.

Хотелось бы использовать логрег из sklearn, но ему нельзя посылать континуальные таргеты...

In [15]:
class EMClassifier:
    def __init__(self, w=None, lr=25, n_iter=10, batch_size=10000, verbose=1):
        self.w = w
        self.lr = lr
        self.n_iter = n_iter
        self.batch_size = batch_size
        self.verbose = verbose
        
    def _add_intercept(self, X):
        """ add pseuso-feature for intercept """
        return scipy.sparse.hstack((np.ones((X.shape[0], 1)), X), format='csr')
    
    def _init_w(self, dim):
        self.w = np.random.randn(dim)
        
    def E_step(self, data, preds):
        """ recalculation of prediction according to team power """
        team_power = pd.DataFrame({'team_id': data['team_id'],
                                  'question_id': data['question_id'], 
                                  'team_power': 1 - preds})
        team_power = team_power.groupby(['team_id', 'question_id'])\
                            .agg({'team_power': 'prod'}).reset_index()
        team_power['team_power'] = 1 - team_power['team_power']
        team_power = data[['team_id', 'question_id']].merge(team_power)
        y = np.clip(preds / team_power['team_power'], 0, 1).values
        y[data['answer'] == 0] = 0  # return wrong answers to zeros
        return y
        
    def M_step(self, X, y):
        """ minibatch GD """    
        min_loss = np.inf
        indices = np.arange(X.shape[0])
        for _ in range(100):
            indices = np.random.permutation(indices)
            for batch_idx in np.array_split(indices, len(indices) // self.batch_size):
                x_batch, y_batch = X[batch_idx], y[batch_idx]
                grad = x_batch.T.dot(self.predict(x_batch) - y_batch) / len(y_batch)
                self.w -= self.lr * grad
                
            cur_loss = self.log_loss(y, self.predict(X))
            if min_loss - cur_loss < 1e-6:
                break
                
            min_loss = cur_loss
                
    def fit(self, X_train, train_data):
        X_train = self._add_intercept(X_train)
        if self.w is None or len(self.w) != X_train.shape[1]:
            self._init_w(X_train.shape[1])
        
        for _ in tqdm.tqdm(range(self.n_iter)): 
            preds = self.predict(X_train)
            y = self.E_step(train_data, preds)
            self.M_step(X_train, y)
            
            if self.verbose:
                compute_scores(train_data, self.predict(X_train))
                         
    def predict(self, X):
        if self.w is None:
            raise ValueError('Model is not fitted yet!')
        if len(self.w) != X.shape[1]:
            X = self._add_intercept(X)
        return scipy.special.expit(X.dot(self.w))  # sigmoid

    @staticmethod
    def log_loss(y_true, y_pred):
        return - np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [16]:
# initialization by already fitted model
w_start = np.hstack([logreg.intercept_, logreg.coef_[0]])
em_classifier = EMClassifier(w_start, n_iter=10, verbose=0)

em_classifier.fit(X_train, train_data)

100%|██████████| 10/10 [08:16<00:00, 49.70s/it]


In [18]:
y_pred = em_classifier.predict(X_test)
compute_scores(test_data, y_pred)

Корреляция Спирмана:  0.811
Корреляция Кендалла:  0.642


Вот. Даже за 10 эпох качество слегка улучшилось

## 5. А что там с вопросами?
- Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

первый вес нашей модели - это интерсепт

In [ ]:
## previously calculated
# feature_names = ohe.get_feature_names(['player_id', 'question_id'])

em_importance = pd.DataFrame({
    'feature': feature_names,
    'value': em_classifier.w[1:],
})
question_level = em_importance[
    em_importance.feature.str.startswith('question')]

_f = lambda s: int(s.split('_')[-1])
question_level['question_id'] = question_level.feature.apply(_f)

data_question = train_data.groupby("question_id")\
                            .agg({'game_id': 'min'}).reset_index()

question_level = pd.merge(question_level, data_question, on='question_id')
games_rating = question_level.groupby('game_id')['value'].mean().sort_values()

games_rating = pd.merge(
    games_rating, data_tours[['id', 'name']], 
    left_on='game_id', right_on='id'
)

### Выведем названия 10 самых трудных турниров и 10 самый легких:

In [50]:
pd.concat([games_rating.head(10), games_rating.tail(10)])

,value,id,name
0,-4.536236,6149,Чемпионат Санкт-Петербурга. Первая лига
1,-2.148835,5928,Угрюмый Ёрш
2,-1.998377,5684,Синхрон высшей лиги Москвы
3,-1.871278,5159,Первенство правого полушария
4,-1.774535,6101,Воображаемый музей
5,-1.701694,5025,Кубок городов
6,-1.616893,5587,Записки охотника
7,-1.579695,5946,Чемпионат Мира. Этап 3. Группа В
8,-1.534003,5083,Ускользающая сова
9,-1.501268,5465,Чемпионат России


Первые и высшие лиги посложнее будут, а школьные, стартовые и малые лиги полегче. Результат удовлетворительный!